In [2]:
from datasets.UPV.load_upv import load_upv_dataset, codes
import pandas as pd

upv = load_upv_dataset()
upv.columns

Loaded UPV dataset from UPV_allURLs.csv
Loading html files...
Removing rows without html: [6668]


Extracting articles from HTML:   0%|          | 0/408 [00:00<?, ?it/s]

Index(['Medio', 'ID_Medio_Crono', 'Título desmentido', 'URL',
       'Titular enlazado', '¿Periodo de campaña?', '¿Qué campaña?',
       '¿Campaña?', 'Único', 'Duplicado misma campaña', 'Corpus',
       'Numeración duplicados', 'ID Corpus', 'ID ALE', '1. Espacio',
       '2. Texto', '3. Multimedia', '4. Elemento desinformador',
       '5.  Tipo multimedia', '6. Alteración multimedia', '7. Cuenta emisora',
       '8. Fuente', '9. Protagonista', '10. Atributo', '11. Macro tema',
       '12. Populismo', '13. Ataque', '14. Tipo de ataque', 'html', 'article',
       'text'],
      dtype='object')

In [ ]:
# spotted an anotation error
upv[(upv['2. Texto'] == 'Sí') & (upv['3. Multimedia'] == 'No') &
    (upv['4. Elemento desinformador'] == 'Elemento multimedia')]['URL'].values

array(['https://maldita.es/malditobulo/20210203/el-pais-fiscalia-anticorrupcion-m-rajoy-papeles-barcenas/'],
      dtype=object)

In [ ]:
upv['text'].str.split().str.len().describe()

count     408.000000
mean      805.102941
std       457.946092
min       129.000000
25%       453.500000
50%       724.500000
75%      1116.000000
max      3417.000000
Name: text, dtype: float64

In [ ]:
# study html and look for links
from bs4 import BeautifulSoup

BeautifulSoup(upv.html.iloc[0])

In [34]:
upv_anotations = ['1. Espacio',
                  '2. Texto', '3. Multimedia', '4. Elemento desinformador',
                  '5.  Tipo multimedia', '6. Alteración multimedia', '7. Cuenta emisora',
                  '8. Fuente', '9. Protagonista', '10. Atributo', '11. Macro tema',
                  '12. Populismo', '13. Ataque', '14. Tipo de ataque']

# select short columns
upv_anotated_counts = upv[upv_anotations].count()

short_cols = upv_anotated_counts[
    upv_anotated_counts < upv_anotated_counts.max()].index

print(upv_anotated_counts)

1. Espacio                   408
2. Texto                     230
3. Multimedia                230
4. Elemento desinformador    230
5.  Tipo multimedia          230
6. Alteración multimedia     230
7. Cuenta emisora            408
8. Fuente                    408
9. Protagonista              408
10. Atributo                 408
11. Macro tema               408
12. Populismo                408
13. Ataque                   408
14. Tipo de ataque           408
dtype: int64


In [35]:
upv['text']

0        MALDITO\nDATO\n\nEs falso que Vox esté en el G...
1        Por María Blanco05 noviembre 2019 |   2  min l...
2        Por Lorena Baeza08 noviembre 2019 |   < 1  min...
3        MALDITO\nDATO\n\nEs falso que el PP gobierne "...
4        MALDITO\nDATO\n\nEs falso que Sánchez lleve "c...
                               ...                        
11704    MALDITO\nBULO\n\nNo, esta foto no muestra que ...
11707    MALDITO\nBULO\n\nCuidado con el audio manipula...
11709    MALDITO\nBULO\n\nNo, no es cierto que este ver...
11713    MALDITO\nBULO\n\n¿Qué sabemos sobre si “Sánche...
11716    MALDITO\nBULO\n\nNo, PACMA no ha quedado en pr...
Name: text, Length: 408, dtype: object

In [29]:
upv['word_len'] = upv['text'].str.split().str.len()
for feat in ['¿Qué campaña?', 'Medio']:
    table = upv.groupby(feat)['word_len'].describe(percentiles=[0.5]).apply(
        pd.to_numeric, downcast='integer').reindex(upv[feat].unique())
    display(table)
    print(table.to_latex(float_format="%.0f",
          caption=f"Article length in words by {feat}"))

,count,mean,std,min,50%,max
¿Qué campaña?,,,,,,
Gen19,92,510.086957,296.411158,129,442.0,1609
Gal-Eus20,41,687.170732,378.482358,158,664.0,1538
Cat21,85,766.188235,434.938980,157,665.0,1823
Mad21,82,1020.609756,554.781945,162,952.5,3417
CyL22,46,979.347826,409.754464,252,1018.5,1757
And22,62,959.903226,358.859245,222,956.5,2327


\begin{table}
\caption{Article length in words by ¿Qué campaña?}
\begin{tabular}{lrrrrrr}
\toprule
 & count & mean & std & min & 50% & max \\
¿Qué campaña? &  &  &  &  &  &  \\
\midrule
Gen19 & 92 & 510 & 296 & 129 & 442 & 1609 \\
Gal-Eus20 & 41 & 687 & 378 & 158 & 664 & 1538 \\
Cat21 & 85 & 766 & 435 & 157 & 665 & 1823 \\
Mad21 & 82 & 1021 & 555 & 162 & 952 & 3417 \\
CyL22 & 46 & 979 & 410 & 252 & 1018 & 1757 \\
And22 & 62 & 960 & 359 & 222 & 956 & 2327 \\
\bottomrule
\end{tabular}
\end{table}



,count,mean,std,min,50%,max
Medio,,,,,,
Maldito dato,36,516.527778,314.665308,144,420,1393
Newtral Fact-checks,109,1064.174312,372.077995,342,1124,1823
Maldito bulo,194,679.484536,446.321386,129,581,2955
Verificat,26,1059.038462,607.867188,323,1040,3417
Newtral fakes,36,723.055556,252.249870,318,715,1273
Efe Verifica,7,1215.285714,303.893465,928,1200,1743


\begin{table}
\caption{Article length in words by Medio}
\begin{tabular}{lrrrrrr}
\toprule
 & count & mean & std & min & 50% & max \\
Medio &  &  &  &  &  &  \\
\midrule
Maldito dato & 36 & 517 & 315 & 144 & 420 & 1393 \\
Newtral Fact-checks & 109 & 1064 & 372 & 342 & 1124 & 1823 \\
Maldito bulo & 194 & 679 & 446 & 129 & 581 & 2955 \\
Verificat & 26 & 1059 & 608 & 323 & 1040 & 3417 \\
Newtral fakes & 36 & 723 & 252 & 318 & 715 & 1273 \\
Efe Verifica & 7 & 1215 & 304 & 928 & 1200 & 1743 \\
\bottomrule
\end{tabular}
\end{table}



In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.pipeline import Pipeline, make_pipeline
from stopwordsiso import stopwords
import numpy as np

campañas = upv['¿Qué campaña?'].unique()

stop_words = list(stopwords('es'))

tfidf_vectorizer = TfidfVectorizer(
    stop_words=stop_words, use_idf=True, sublinear_tf=True, ngram_range=(1, 2), min_df=3, max_df=0.8)

clf = RandomForestClassifier()

pipe = make_pipeline(tfidf_vectorizer, clf)

results = {}
for i in range(1, len(campañas)):
    campañas_train, campañas_test = campañas[:i], campañas[i:]
    train_test_s = f'{len(campañas_train)}/{len(campañas_test)}'
    results[train_test_s] = {}
    print(f'training on: {campañas_train}, testing on: {campañas_test}')
    for anotation in upv_anotations:
        selection = upv[upv[anotation].notnull()]
        data = {
            'train': selection[selection['¿Qué campaña?'].isin(campañas_train)],
            'dev': selection[selection['¿Qué campaña?'].isin(campañas_test)]}
        pipe.fit(data['train']['text'], data['train'][anotation])
        predicted = pipe.predict(data['dev']['text'])
        accuracy = np.mean(predicted == data['dev'][anotation])
        try:
            f1 = f1_score(y_true=data['dev'][anotation], y_pred=predicted)
        except:
            f1 = f1_score(y_true=data['dev'][anotation],
                          y_pred=predicted, average='macro')
        # print('dimensión', anotation)
        # print(f'accuracy: {accuracy}', f'f1: {f1}')
        results[train_test_s][anotation] = {'accuracy': accuracy, 'f1': f1}

training on: ['Gen19'], testing on: ['Gal-Eus20' 'Cat21' 'Mad21' 'CyL22' 'And22']
training on: ['Gen19' 'Gal-Eus20'], testing on: ['Cat21' 'Mad21' 'CyL22' 'And22']


/home/jreyes/HAMiSoN/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


training on: ['Gen19' 'Gal-Eus20' 'Cat21'], testing on: ['Mad21' 'CyL22' 'And22']


/home/jreyes/HAMiSoN/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


training on: ['Gen19' 'Gal-Eus20' 'Cat21' 'Mad21'], testing on: ['CyL22' 'And22']


/home/jreyes/HAMiSoN/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


training on: ['Gen19' 'Gal-Eus20' 'Cat21' 'Mad21' 'CyL22'], testing on: ['And22']


/home/jreyes/HAMiSoN/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1760: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [59]:
for metric in ['accuracy', 'f1']:
    table = pd.DataFrame.from_dict(results).map(lambda x: x[metric])
    table = table.rename_axis(
        columns=f'{metric} by campaigns train/test',
        index='anotation')
    display(table)
    print(table.to_latex(float_format="%.2f"))

accuracy by campaigns train/test,1/5,2/4,3/3,4/2,5/1
anotation,,,,,
1. Espacio,0.876582,0.901818,0.931579,0.925926,0.903226
2. Texto,0.988827,1.000000,1.000000,1.000000,1.000000
3. Multimedia,0.703911,0.671053,0.654206,0.760000,0.827586
4. Elemento desinformador,0.636872,0.651316,0.635514,0.680000,0.620690
5. Tipo multimedia,0.396648,0.361842,0.504673,0.400000,0.310345
6. Alteración multimedia,0.486034,0.427632,0.439252,0.480000,0.517241
7. Cuenta emisora,0.680380,0.687273,0.742105,0.675926,0.709677
8. Fuente,0.743671,0.781818,0.821053,0.879630,0.854839
9. Protagonista,0.360759,0.396364,0.489474,0.555556,0.548387


\begin{tabular}{lrrrrr}
\toprule
accuracy by campaigns train/test & 1/5 & 2/4 & 3/3 & 4/2 & 5/1 \\
anotation &  &  &  &  &  \\
\midrule
1. Espacio & 0.88 & 0.90 & 0.93 & 0.93 & 0.90 \\
2. Texto & 0.99 & 1.00 & 1.00 & 1.00 & 1.00 \\
3. Multimedia & 0.70 & 0.67 & 0.65 & 0.76 & 0.83 \\
4. Elemento desinformador & 0.64 & 0.65 & 0.64 & 0.68 & 0.62 \\
5.  Tipo multimedia & 0.40 & 0.36 & 0.50 & 0.40 & 0.31 \\
6. Alteración multimedia & 0.49 & 0.43 & 0.44 & 0.48 & 0.52 \\
7. Cuenta emisora & 0.68 & 0.69 & 0.74 & 0.68 & 0.71 \\
8. Fuente & 0.74 & 0.78 & 0.82 & 0.88 & 0.85 \\
9. Protagonista & 0.36 & 0.40 & 0.49 & 0.56 & 0.55 \\
10. Atributo & 0.75 & 0.75 & 0.71 & 0.67 & 0.66 \\
11. Macro tema & 0.59 & 0.66 & 0.64 & 0.68 & 0.71 \\
12. Populismo & 0.54 & 0.56 & 0.57 & 0.58 & 0.61 \\
13. Ataque & 0.60 & 0.60 & 0.63 & 0.56 & 0.58 \\
14. Tipo de ataque & 0.53 & 0.53 & 0.51 & 0.44 & 0.47 \\
\bottomrule
\end{tabular}



f1 by campaigns train/test,1/5,2/4,3/3,4/2,5/1
anotation,,,,,
1. Espacio,0.590658,0.608158,0.627771,0.625735,0.612428
2. Texto,0.497191,0.000000,0.000000,0.000000,0.000000
3. Multimedia,0.036364,0.137931,0.097561,0.000000,0.000000
4. Elemento desinformador,0.778157,0.774468,0.757764,0.794872,0.755556
5. Tipo multimedia,0.217567,0.237594,0.341315,0.291097,0.223611
6. Alteración multimedia,0.366146,0.310036,0.330483,0.334091,0.374504
7. Cuenta emisora,0.166022,0.170053,0.213472,0.196856,0.309253
8. Fuente,0.462848,0.474806,0.533544,0.799714,0.798493
9. Protagonista,0.058168,0.128781,0.152270,0.198230,0.241515


\begin{tabular}{lrrrrr}
\toprule
f1 by campaigns train/test & 1/5 & 2/4 & 3/3 & 4/2 & 5/1 \\
anotation &  &  &  &  &  \\
\midrule
1. Espacio & 0.59 & 0.61 & 0.63 & 0.63 & 0.61 \\
2. Texto & 0.50 & 0.00 & 0.00 & 0.00 & 0.00 \\
3. Multimedia & 0.04 & 0.14 & 0.10 & 0.00 & 0.00 \\
4. Elemento desinformador & 0.78 & 0.77 & 0.76 & 0.79 & 0.76 \\
5.  Tipo multimedia & 0.22 & 0.24 & 0.34 & 0.29 & 0.22 \\
6. Alteración multimedia & 0.37 & 0.31 & 0.33 & 0.33 & 0.37 \\
7. Cuenta emisora & 0.17 & 0.17 & 0.21 & 0.20 & 0.31 \\
8. Fuente & 0.46 & 0.47 & 0.53 & 0.80 & 0.80 \\
9. Protagonista & 0.06 & 0.13 & 0.15 & 0.20 & 0.24 \\
10. Atributo & 0.29 & 0.28 & 0.28 & 0.27 & 0.27 \\
11. Macro tema & 0.35 & 0.45 & 0.41 & 0.45 & 0.41 \\
12. Populismo & 0.26 & 0.30 & 0.31 & 0.42 & 0.47 \\
13. Ataque & 0.60 & 0.67 & 0.67 & 0.57 & 0.58 \\
14. Tipo de ataque & 0.13 & 0.13 & 0.14 & 0.13 & 0.13 \\
\bottomrule
\end{tabular}



In [22]:
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from nltk.corpus.stopwords import words as stopwords
from stopwordsiso import stopwords
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


stop_words = list(stopwords('es'))

tfidf_vectorizer = TfidfVectorizer(
    stop_words=stop_words, use_idf=True, sublinear_tf=True, ngram_range=(1, 2))

campañas_all = upv.groupby('¿Qué campaña?').agg({'text': ' '.join})
campañas_all

tfidf_matrix = tfidf_vectorizer.fit_transform(campañas_all['text'])

# Step 3: Create DataFrames
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(),
                        columns=tfidf_vectorizer.get_feature_names_out())
tfidf_df

tfidf_df.index = tfidf_df.index.map(lambda x: x+1)
tfidf_df.index = tfidf_df.index.map(codes['¿Qué campaña?']['map_values'])
tfidf_df

,00,00 00,00 06,00 20,00 21,00 22,00 acto,00 admiten,00 atenció,00 casos,...,útil eficaz,útil ingreso,útil manejo,útil reduzcan,𝔊𝔞𝔪𝔟𝔯𝔦𝔫𝔲𝔰,𝔊𝔞𝔪𝔟𝔯𝔦𝔫𝔲𝔰 svq666,𝔖𝔩𝔞𝔳𝔦𝔨,𝔖𝔩𝔞𝔳𝔦𝔨 𝔊𝔞𝔪𝔟𝔯𝔦𝔫𝔲𝔰,𝗕𝗨𝗟𝗢,𝗕𝗨𝗟𝗢 respecte
Gen19,0.006493,0.004296,0.000000,0.004296,0.005239,0.000000,0.005239,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.005239,0.005239,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Gal-Eus20,0.005232,0.000000,0.004050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.004939,0.000000,0.000000,0.004939,0.004939,0.004939,0.004939,0.000000,0.000000
Cat21,0.002741,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Mad21,0.003394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.007645,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
CyL22,0.009459,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
And22,0.008249,0.006116,0.003612,0.006116,0.000000,0.004405,0.000000,0.004405,0.004405,0.004405,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.004405,0.004405


In [ ]:
tfidf_df.idxmax(axis=1)
for campaña in tfidf_df.index:
    print('campaña:', campaña)
    # display(tfidf_df.loc[campaña].nlargest(12))

    # Create and generate a word cloud image:
    # wordcloud = WordCloud().generate(text)
    wordcloud = WordCloud().generate_from_frequencies(
        tfidf_df.loc[campaña].to_dict()
    )

    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig(f'./datasets/UPV/wordclouds/wordcloud-{campaña}.png')
    plt.show()

In [ ]:
for campaña, group in upv_anotated.groupby('¿Qué campaña?'):
    print("Campaña", campaña)

    all_text = upv_anotated.text[0]

    # Create and generate a word cloud image:
    wordcloud = WordCloud().generate(text)

    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()